<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/%EB%82%98%EB%A7%8C%EC%9D%98_%EC%9D%8C%EC%95%85_%EC%83%9D%EC%84%B1%EA%B8%B0_K2024504_%EA%B5%AC%EB%B9%9B%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install --upgrade google-generativeai gradio nest_asyncio

In [37]:
import asyncio
import gradio as gr
import nest_asyncio
import numpy as np
from google import genai
from google.genai import types
import google.generativeai as genai
from google import genai as genai_beta

In [38]:
nest_asyncio.apply()  # Colab의 asyncio 충돌 방지

# 파일 저장 도구
import tempfile
import os

async def generate_music(api_key, prompt_text="minimal techno", bpm=90, temperature=1.0,
                         brightness=0.5, density=0.7, guidance=4.0):
    from google import genai as genai_beta  # 내부 import로 안정성 확보
    client = genai_beta.Client(api_key=api_key, http_options={'api_version': 'v1alpha'})

    audio_chunks = []

    async def receive_audio(session):
        try:
            async for message in session.receive():
                if message.server_content.audio_chunks:
                    chunk = message.server_content.audio_chunks[0].data
                    print("✅ Received chunk of size:", len(chunk))
                    audio_chunks.append(chunk)
                else:
                    print("⚠️ Received message without audio chunks")
                await asyncio.sleep(0.001)
        except Exception as e:
            print("❌ receive_audio() 내부 예외 발생:", e)

    try:
        async with (
            client.aio.live.music.connect(model='models/lyria-realtime-exp') as session,
            asyncio.TaskGroup() as tg,
        ):
            tg.create_task(receive_audio(session))

            await session.set_weighted_prompts(
                prompts=[types.WeightedPrompt(text=prompt_text, weight=1.0)]
            )
            await session.set_music_generation_config(
                config=types.LiveMusicGenerationConfig(
                    bpm=bpm,
                    temperature=temperature,
                    brightness=brightness,
                    density=density,
                    guidance=guidance
                )
            )
            print("▶️ 시작: 음악 생성 요청")
            await session.play()
            await asyncio.sleep(8)
            await session.stop()

    except Exception as e:
        print("❌ 예외 발생:", e)
        return None

    from scipy.io.wavfile import write
    import numpy as np

    raw_bytes = b"".join(audio_chunks)
    waveform = np.frombuffer(raw_bytes, dtype=np.int16)

    if np.all(waveform == 0):
        print("⚠️ 수신된 오디오는 전부 무음입니다. 저장하지 않습니다.")
        return None

    temp_path = "/content/my_music.wav"
    write(temp_path, 24000, waveform)
    print("✅ 오디오 저장 완료:", temp_path)
    return temp_path


In [39]:
def gradio_interface(api_key, prompt_text, bpm, temperature, brightness, density, guidance):
    loop = asyncio.get_event_loop()
    try:
        audio_path = loop.run_until_complete(
            generate_music(api_key, prompt_text, bpm, temperature)
        )

        if audio_path is None or not os.path.exists(audio_path):
            return "❌ 음악 생성 실패"

        file_size = os.path.getsize(audio_path)
        if file_size < 10000:
            return "⚠️ 생성된 오디오가 너무 짧거나 비어 있음"

        print("🎧 생성된 파일:", audio_path, "| 크기:", file_size)
        return audio_path

    except Exception as e:
        print("💥 예외 발생:", str(e))
        return f"❌ 예외 발생: {str(e)}"

In [ ]:
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="🔐 API Key", placeholder="여기에 Google API 키를 입력하세요", type="password"),
        gr.Textbox(label="🎵 Prompt", value="minimal techno"),
        gr.Slider(60, 180, step=1, label="🎚️ BPM (비트)", value=90),
        gr.Slider(0.1, 3.0, step=0.1, label="🌡️ Temperature (창의성)", value=1.0),
        gr.Slider(0.0, 1.0, step=0.1, label="✨ Brightness (밝기)", value=0.5),
        gr.Slider(0.0, 1.0, step=0.1, label="🎼 Density (밀도)", value=0.7),
        gr.Slider(0.0, 6.0, step=0.1, label="📏 Guidance (프롬프트 준수도)", value=4.0),
    ],
    outputs=gr.Audio(type="filepath", label="🎧 Generated Music"),
    title="🎶 Lyria: 나만의 음악 생성기",
    description=(
        "**📝 생성 파라미터 설명:**\n"
        "- 🎚️ **BPM (Beats Per Minute)**: 음악의 속도. 높을수록 빠른 곡, 낮을수록 느린 곡.\n"
        "- 🌡️ **Temperature**: 창의성 정도. 낮으면 반복적이고 안정적, 높으면 다양하고 창의적인 결과.\n"
        "- ✨ **Brightness**: 밝기. 1.0에 가까울수록 경쾌하고 높은 음, 0.0이면 어둡고 낮은 음 위주.\n"
        "- 🎼 **Density**: 음의 밀도. 높으면 더 풍부하고 복잡한 곡, 낮으면 단순한 느낌.\n"
        "- 📏 **Guidance**: 프롬프트를 얼마나 정확히 따를지 제어. 높을수록 프롬프트에 충실하지만 딱딱할 수 있음.\n"
        "\n**모든 파라미터는 Lyria 모델에게 생성 음악의 스타일을 실시간으로 조정하는 지시 역할을 합니다.**"
    ),
)

iface.launch(debug=True, share=True)

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-161' coro=<AsyncClient.aclose() running at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978>>
/usr/lib/python3.11/asyncio/base_events.py:679: RuntimeWarning: coroutine 'AsyncClient.aclose' was never awaited
  self._ready.clear()
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-193' coro=<AsyncClient.aclose() running at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978>>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-225' coro=<AsyncClient.aclose() running at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978>>


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6341bca3a748017911.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
